In [1]:
import findspark
import pandas as pd
findspark.init()

from pyspark.sql import SparkSession
from pyspark import SparkConf

# for shared metastore (shared across all users)
spark = SparkSession.builder.appName("Identification").config("hive.metastore.uris", "thrift://bialobog:9083", conf=SparkConf()).getOrCreate() \

# for local metastore (your private, invidivual database) add the following config to spark session

spark.sql("USE 2023_04_01")





DataFrame[]

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.metrics import mean_squared_error
from datetime import datetime
from sklearn.linear_model import LinearRegression
from scipy.stats import norm
from pyspark.sql import Row
from pyspark.sql import functions as F
from pyspark.sql.functions import col, to_date, lit, udf, pandas_udf, PandasUDFType, coalesce, \
            month, year, concat, date_format, format_string, last_day, months_between, greatest, least, when, lag, count, desc, expr, log1p
from pyspark.sql.window import Window
from datetime import timedelta
from pyspark.sql.types import*
import random
from tqdm import tqdm
import numpy as np
import pandas as pd
import os
import csv
import math
import pyspark.pandas as ps

/opt/spark/python/pyspark/pandas/__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


In [3]:
def get_all_stocks_prices():
    query = """SELECT s.ticker_region, p.p_date, p.p_price, splits.p_split_date, splits.p_split_factor
                FROM sym_ticker_region s 
                INNER JOIN FF_SEC_COVERAGE c ON c.fsym_id = s.fsym_id
                INNER JOIN sym_coverage sc ON sc.fsym_id = s.fsym_id
                INNER JOIN fp_basic_prices p ON p.fsym_id = sc.fsym_regional_id
                LEFT JOIN fp_basic_splits AS splits ON splits.p_split_date = p.p_date AND p.fsym_id = splits.fsym_id 
                WHERE s.ticker_region LIKE "%-US" AND s.ticker_region NOT LIKE '%.%' AND c.CURRENCY = "USD"
                AND (sc.fref_listing_exchange = "NAS" OR sc.fref_listing_exchange = "NYS")
                AND p.p_date >= '2000-01-01'
                ORDER BY s.ticker_region, p.p_date
                """
    adj = spark.sql(query)
    
    window_spec = Window.partitionBy('ticker_region').orderBy(col('p_date').desc())
    adj = adj.withColumn("cum_split_factor", lit(0.0))  # placeholders
    adj = adj.withColumn("cum_split_factor", when(col('p_date') == col('p_split_date'), col('p_split_factor')).otherwise(lit(1.0)))
    adj = adj.withColumn("cum_split_factor", lit(1.0) * lag("cum_split_factor").over(window_spec)).fillna(1.0)

    # Calculate split-adjusted price
    adj = adj.withColumn("split_adj_price", col("p_price") * col("cum_split_factor"))

    # Sort the DataFrame
    adj = adj.sort(col('ticker_region').asc(), col('p_date').asc())

    # Drop unnecessary columns
    # columns_to_drop = ["fsym_id", "p_split_date", 'p_split_factor']
    # adj = adj.drop(*columns_to_drop)
    
    adj.filter(col('ticker_region')=='AAPL-US').show()
    

    
#get_all_stocks_prices()
    

In [4]:
def get_stock_price_weekly(ticker):
    # Suppress the progress message from yfinance
    temp_df = yf.download(ticker, start='2000-01-01', end='2023-03-31', progress=False)
    if temp_df.empty:
        print("No data available for the specified date range.")
        return None
    print(temp_df['Adj Close'].pct_change())

#get_stock_price_weekly('AAPL')

In [5]:
# adj = spark.sql("""SELECT s.fsym_id, str.ticker_region, sc.proper_name, p.p_date, p.p_price,
#                 splits.p_split_date, splits.p_split_factor

# FROM sym_entity_sector AS e

# INNER JOIN fp_sec_entity AS s ON e.factset_entity_id = s.factset_entity_id
# INNER JOIN sym_coverage AS sc ON s.fsym_id = sc.fsym_security_id 
# INNER JOIN fp_basic_prices AS p ON sc.fsym_regional_id = p.fsym_id
# INNER JOIN factset_sector_map AS sm ON e.sector_code = sm.factset_sector_code
# INNER JOIN ff_basic_qf AS ff ON p.fsym_id = ff.fsym_id
# INNER JOIN ff_advanced_der_af adf ON p.fsym_id = adf.fsym_id
# LEFT JOIN sym_ticker_region str ON str.fsym_id = sc.fsym_regional_id
# LEFT JOIN fp_basic_dividends AS divs ON divs.p_divs_exdate = p.p_date AND p.fsym_id = divs.fsym_id
# LEFT JOIN fp_basic_splits AS splits ON splits.p_split_date = p.p_date AND p.fsym_id = splits.fsym_id 

# WHERE sc.fsym_security_id IS NOT NULL
#         AND sc.fref_security_type IS NOT NULL AND sc.fref_security_type = 'SHARE'
#         AND sc.currency IS NOT NULL AND sc.currency = 'USD'
#         AND s.factset_entity_id IS NOT NULL
#         AND p.p_date IS NOT NULL
#         AND (sc.fref_listing_exchange = 'NAS' OR sc.fref_listing_exchange = 'NYS')
#         AND p.p_date >= '2000-01-01'
#         AND str.ticker_region = 'AAPL-US'
# ORDER BY s.fsym_id ASC, p_date DESC
#             """)


# adj = adj.withColumn("temp_cum_split_factor", when(adj.p_date==adj.p_split_date, lit(adj.p_split_factor)).otherwise(lit(1.0)))
# adj = adj.withColumn("div_split_factor", lit(0.0)) # placeholders
# adj = adj.withColumn("cum_split_factor", lit(0.0)) # placeholders
# adj = adj.withColumn("split_temp_i", lit(0)) # placeholders - for ordering purposes

# # creating udf to calculate cumulative split factor
# @pandas_udf(adj.schema, FloatType(), PandasUDFType.GROUPED_MAP)
# def calc_product_factor(df1):
#     """
#     Calculates the cumulative split factor for each company based on unique fsym_id's,
#     for both the price split and the dividend split.
#     The data MUST be sorted within the function itself (no orderBy in the function call),
#     and spin_temp_i must be set to i during each iteration of the loop to guarantee
#     proper sorting - without these safeguards, the function is applied non-sequentially
#     to the data.
#     """
#     df1 = df1.sort_values(by='p_date', ascending=False)
#     for i in range(0, len(df1)):
#         df1.loc[i, 'split_temp_i'] = i
#         if i == 0:
#             df1.loc[i, 'cum_split_factor'] = 1.0
#             df1.loc[i, 'div_split_factor'] = 1.0
#             continue
#         df1.loc[i-1, 'div_split_factor'] = df1.loc[i-1, 'cum_split_factor'] * df1.loc[i-1, 'temp_cum_split_factor']
#         df1.loc[i, 'cum_split_factor'] = df1.loc[i-1, 'cum_split_factor'] * df1.loc[i-1, 'temp_cum_split_factor']
#     return df1

# adj = adj.groupBy('fsym_id').apply(calc_product_factor)
# adj = adj.withColumn("split_adj_price", (adj.p_price*adj.cum_split_factor))

# adj.filter(col('ticker_region')=='AAPL-US').orderBy('fsym_id', 'p_date').show()

In [6]:

def get_all_stocks_df():
    query = f"""SELECT s.ticker_region, s.fsym_id,p.p_date, p.p_price, splits.p_split_date, splits.p_split_factor, ms.p_com_shs_out 
                FROM sym_ticker_region s 
                LEFT JOIN FF_SEC_COVERAGE c ON c.fsym_id = s.fsym_id
                LEFT JOIN sym_coverage sc ON sc.fsym_id = s.fsym_id
                INNER JOIN fp_basic_prices p ON p.fsym_id = sc.fsym_regional_id
                LEFT JOIN fp_basic_splits AS splits ON splits.p_split_date = p.p_date AND p.fsym_id = splits.fsym_id
                LEFT JOIN (SELECT sf.fsym_id, mp.price_date, sf.p_com_shs_out, sf.p_date AS shares_hist_date
                                        FROM fp_basic_shares_hist AS sf
                                        JOIN (SELECT p2.fsym_id, p2.p_date AS price_date, max(h.p_date) AS max_shares_hist_date
                                                FROM fp_basic_prices AS p2
                                                JOIN fp_basic_shares_hist AS h ON h.p_date <= p2.p_date AND p2.fsym_id = h.fsym_id
                                                GROUP BY p2.fsym_id, p2.p_date)
                                        mp ON mp.fsym_id = sf.fsym_id AND mp.max_shares_hist_date = sf.p_date)
                            ms ON ms.fsym_id = p.fsym_id AND ms.price_date = p.p_date
                WHERE s.ticker_region LIKE "%-US" AND s.ticker_region NOT LIKE '%.%' AND c.CURRENCY = "USD" 
                AND ms.p_com_shs_out IS NOT NULL
                AND (sc.fref_listing_exchange = "NAS" OR sc.fref_listing_exchange = "NYS") AND p.p_date >= '2000-01-01'
                ORDER BY s.fsym_id, p.p_date"""

    

    adj = spark.sql(query)
    #adj.show()
    adj = adj.withColumn("temp_cum_split_factor", when(adj.p_date==adj.p_split_date, lit(adj.p_split_factor)).otherwise(lit(1.0)))
    #adj = adj.withColumn("div_split_factor", lit(0.0)) # placeholders
    adj = adj.withColumn("cum_split_factor", lit(0.0)) # placeholders
    #adj = adj.withColumn("split_temp_i", lit(0)) # placeholders - for ordering purposes

    window_spec = Window.partitionBy('fsym_id').orderBy(F.desc('p_date'))

    # Calculate cumulative split factor and dividend split factor
    #adj = adj.withColumn('split_temp_i', F.row_number().over(window_spec) - 1)
    adj = adj.withColumn('cum_split_factor_no_lag', 
                        F.exp(F.sum(F.log('temp_cum_split_factor')).over(window_spec)))

    # Apply lag to the calculated cumulative split factor
    adj = adj.withColumn('cum_split_factor', 
                        F.when(F.row_number().over(window_spec) == 1, 1.0)
                        .otherwise(F.lag('cum_split_factor_no_lag', default=1.0).over(window_spec)))

    adj = adj.withColumn('split_adj_price', adj.p_price * adj.cum_split_factor)
    
    adj = adj.withColumn('year', F.year('p_date'))
    
    adj =adj.withColumn('week_of_year', F.weekofyear('p_date'))

    window_spec = Window.partitionBy('fsym_id', 'year', 'week_of_year').orderBy(col('p_date').desc())

    adj = adj.withColumn('row_num', F.row_number().over(window_spec))

    adj = adj.filter(col('row_num') == 1)
    
    
    # adj_pd = adj.toPandas()
    # adj_pd['p_date'] = pd.to_datetime(adj_pd['p_date'])

    # plt.figure(figsize=(15, 5))
    # plt.plot(adj_pd['p_date'], adj_pd['split_adj_price'], label='AAPL adjusted')
    # plt.plot(adj_pd['p_date'], adj_pd['p_price'], label='AAPL unadjusted')
    # plt.legend()
    # plt.show()


    adj = adj.drop('temp_cum_split_factor', 'cum_split_factor', 'cum_split_factor_no_lag', 'row_num', 'p_split_date', 'p_split_factor')
    adj = adj.withColumn('Implosion_Start_Date', F.lit(None).cast(DateType()))
    adj = adj.withColumn('Implosion_End_Date', F.lit(None).cast(DateType()))
    adj = adj.withColumn('Market_Value', col('split_adj_price') * col('p_com_shs_out'))
    
    window_spec2 = Window().partitionBy('fsym_id').orderBy('p_date').rowsBetween(-52, Window.currentRow)
    adj = adj.withColumn('rolling_mean', F.avg('split_adj_price').over(window_spec2))
    window_spec3 = Window().partitionBy('fsym_id', 'year').orderBy('p_date')
    adj = adj.withColumn('mv_per_year', F.avg('Market_Value').over(window_spec3))
    
    
    adj.orderBy('fsym_id', 'p_date').show()
    
    
    
    @pandas_udf(adj.schema, FloatType(), PandasUDFType.GROUPED_MAP)
    def find_implosion(df1):
        fsym_id = df1.loc[0,'fsym_id']
        df1 = df1.sort_values(by='p_date')
        i = 52
        price_drop_thresh = -0.6
        period_thresh = 104
        imp_dates = []
        while i < len(df1):
            current_date = df1.loc[i, 'p_date']
            current_price = df1.loc[i, 'split_adj_price']
            mean_price = df1.loc[i, 'rolling_mean']
            if (current_price - mean_price)/mean_price < price_drop_thresh:
                j = i
                start_price=current_price
                j+=1
                imp_period=0
                while j < len(df1) and (df1.loc[j, 'split_adj_price'] - start_price) / start_price <= 0.2:
                    imp_period+=1
                    j+=1
                if imp_period > period_thresh:
                    imp_dates.append((current_date, df1.loc[i+imp_period, 'p_date']))
                i+=imp_period
            i+=1
        if len(imp_dates)>0:
            df1.loc[df1['fsym_id']==fsym_id, 'Implosion_Start_Date'] = imp_dates[0][0]
            df1.loc[df1['fsym_id']==fsym_id, 'Implosion_End_Date'] = imp_dates[0][1]
        
        return df1
    
    @pandas_udf(adj.schema, FloatType(), PandasUDFType.GROUPED_MAP)
    def find_implosion_mv(df1):
        fsym_id = df1.loc[0,'fsym_id']
        df1 = df1.sort_values(by='p_date')
        i = 52
        mv_drop_thresh = -0.6
        period_thresh = 104
        imp_dates = []
        while i < len(df1):
            current_date = df1.loc[i, 'p_date']
            current_mv = df1.loc[i, 'Market_Value']
            prev_year_mv = df1.loc[i, 'mv_per_year']
            if (current_mv - prev_year_mv)/prev_year_mv < mv_drop_thresh:
                j = i
                start_mv = current_mv
                j+=1
                imp_period=0
                while j < len(df1) and (df1.loc[j, 'Market_Value'] - start_mv) / start_mv <= 0.2:
                    imp_period+=1
                    j+=1
                if imp_period > period_thresh:
                    imp_dates.append((current_date, df1.loc[i+imp_period, 'p_date']))
                i+=imp_period
            i+=1
        if len(imp_dates)>0:
            df1.loc[df1['fsym_id']==fsym_id, 'Implosion_Start_Date'] = imp_dates[0][0]
            df1.loc[df1['fsym_id']==fsym_id, 'Implosion_End_Date'] = imp_dates[0][1]
        
        return df1

    

    # adj = adj.filter(col('ticker_region')=='AAPL-US').orderBy('fsym_id', 'p_date')
    # find_implosion2(ps.DataFrame(adj))
    # adj = adj.groupBy('fsym_id').apply(find_implosion)
    # result_df = adj.groupBy('ticker_region').agg(F.first('Implosion_Start_Date').alias('Implosion_Start_Date'), F.first('Implosion_End_Date').alias('Implosion_End_Date'))
    # result_df.show()
    # result_df.toPandas().to_csv('imploded_stocks3.csv')
    
    adj = adj.groupBy('fsym_id').apply(find_implosion_mv)
    adj.show(5)
    result_df = adj.groupBy('ticker_region').agg(F.first('Implosion_Start_Date').alias('Implosion_Start_Date'), F.first('Implosion_End_Date').alias('Implosion_End_Date'))
    result_df.toPandas().to_csv('imploded_stocks_mv.csv')
    
    

    #adj.show()
    
    
    
    
    
get_all_stocks_df()

+-------------+--------+----------+-------+-------------+---------------+----+------------+--------------------+------------------+------------------+------------------+------------------+
|ticker_region| fsym_id|    p_date|p_price|p_com_shs_out|split_adj_price|year|week_of_year|Implosion_Start_Date|Implosion_End_Date|      Market_Value|      rolling_mean|       mv_per_year|
+-------------+--------+----------+-------+-------------+---------------+----+------------+--------------------+------------------+------------------+------------------+------------------+
|      PBFX-US|B00FG1-R|2014-05-09|  27.68|    15886.553|          27.68|2014|          19|                null|              null|439739.78703999997|             27.68|439739.78703999997|
|      PBFX-US|B00FG1-R|2014-05-16|  26.92|      15887.0|          26.92|2014|          20|                null|              null|427678.04000000004|              27.3|      433708.91352|
|      PBFX-US|B00FG1-R|2014-05-23|  26.74|      15887.

/opt/spark/python/pyspark/sql/pandas/group_ops.py:103: UserWarning: It is preferred to use 'applyInPandas' over this API. This API will be deprecated in the future releases. See SPARK-28264 for more details.
  warnings.warn(


+-------------+--------+----------+-------+-------------+---------------+----+------------+--------------------+------------------+------------+------------+-----------+
|ticker_region| fsym_id|    p_date|p_price|p_com_shs_out|split_adj_price|year|week_of_year|Implosion_Start_Date|Implosion_End_Date|Market_Value|rolling_mean|mv_per_year|
+-------------+--------+----------+-------+-------------+---------------+----+------------+--------------------+------------------+------------+------------+-----------+
|      BSFC-US|B06303-R|2020-02-21|  501.0|       6074.0|          501.0|2020|           8|                null|              null|   3043074.0|       501.0|  3043074.0|
|      BSFC-US|B06303-R|2020-02-28|  501.0|      17558.0|          501.0|2020|           9|                null|              null|   8796558.0|       501.0|  5919816.0|
|      BSFC-US|B06303-R|2020-03-06|  501.0|      17558.0|          501.0|2020|          10|                null|              null|   8796558.0|      

In [7]:
def plot_prices(imp_df, pic_name):
    imp_df.createOrReplaceTempView("temp_table")
    
    query2 = f"""SELECT t.ticker_region, s.fsym_id,p.p_date, p.p_price, splits.p_split_date, splits.p_split_factor, 
                divs.p_divs_exdate, divs.p_divs_s_pd, divs.p_divs_pd, ms.p_com_shs_out 
                FROM temp_table t
                LEFT JOIN sym_ticker_region s ON s.ticker_region = t.ticker_region
                LEFT JOIN FF_SEC_COVERAGE c ON c.fsym_id = s.fsym_id
                LEFT JOIN sym_coverage sc ON sc.fsym_id = s.fsym_id
                INNER JOIN fp_basic_prices p ON p.fsym_id = sc.fsym_regional_id
                LEFT JOIN fp_basic_dividends divs ON divs.p_divs_exdate = p.p_date AND p.fsym_id = divs.fsym_id
                LEFT JOIN fp_basic_splits AS splits ON splits.p_split_date = p.p_date AND p.fsym_id = splits.fsym_id
                LEFT JOIN (SELECT sf.fsym_id, mp.price_date, sf.p_com_shs_out, sf.p_date AS shares_hist_date
                                        FROM fp_basic_shares_hist AS sf
                                        JOIN (SELECT p2.fsym_id, p2.p_date AS price_date, max(h.p_date) AS max_shares_hist_date
                                                FROM fp_basic_prices AS p2
                                                JOIN fp_basic_shares_hist AS h ON h.p_date <= p2.p_date AND p2.fsym_id = h.fsym_id
                                                GROUP BY p2.fsym_id, p2.p_date)
                                        mp ON mp.fsym_id = sf.fsym_id AND mp.max_shares_hist_date = sf.p_date)
                            ms ON ms.fsym_id = p.fsym_id AND ms.price_date = p.p_date
                WHERE p.p_date >= '2000-01-01'
                ORDER BY t.ticker_region, p.p_date
                """
    
    adj = spark.sql(query2)
    imp_df = imp_df.toPandas()
    adj.show()
    adj = adj.withColumn("temp_cum_split_factor", when(adj.p_date==adj.p_split_date, lit(adj.p_split_factor)).otherwise(lit(1.0)))
    adj = adj.withColumn("cum_split_factor", lit(0.0))

    window_spec = Window.partitionBy('fsym_id').orderBy(F.desc('p_date'))

    adj = adj.withColumn('cum_split_factor_no_lag', 
                        F.exp(F.sum(F.log('temp_cum_split_factor')).over(window_spec)))

    adj = adj.withColumn('cum_split_factor', 
                        F.when(F.row_number().over(window_spec) == 1, 1.0)
                        .otherwise(F.lag('cum_split_factor_no_lag', default=1.0).over(window_spec)))
    
    adj = adj.withColumn('split_adj_price', adj.p_price * adj.cum_split_factor)
#     adj = adj.withColumn("split_adj_div", (adj.p_divs_pd*adj.cum_split_factor))
    
#     adj = adj.withColumn("div_factor", when(((adj.p_date == adj.p_divs_exdate) & (adj.p_divs_s_pd == 1)), lit(adj.split_adj_div)).otherwise(lit(0.0)))
#     adj = adj.withColumn("temp_cum_spin_factor", when((adj.split_adj_price - adj.div_factor <= 0), lit(1.0)) \
#                      .otherwise(lit((adj.split_adj_price - adj.div_factor)/adj.split_adj_price)))
#     adj = adj.withColumn("cum_spin_factor", lit(0.0))
    
#     adj = adj.withColumn('cum_spin_factor_no_lag', 
#                         F.exp(F.sum(F.log('temp_cum_spin_factor')).over(window_spec)))
    
#     adj = adj.withColumn('cum_spin_factor', 
#                         F.when(F.row_number().over(window_spec) == 1, 1.0)
#                         .otherwise(F.lag('cum_spin_factor_no_lag').over(window_spec)))
    
#     adj = adj.withColumn("adj_price", adj.split_adj_price * adj.cum_spin_factor)
    
    adj = adj.withColumn('Market_Value', col('split_adj_price') * col('p_com_shs_out'))
    
    adj = adj.withColumn('year', F.year('p_date'))
    
    adj =adj.withColumn('week_of_year', F.weekofyear('p_date'))

    window_spec = Window.partitionBy('fsym_id', 'year', 'week_of_year').orderBy(col('p_date').desc())

    adj = adj.withColumn('row_num', F.row_number().over(window_spec))

    adj = adj.filter(col('row_num') == 1).orderBy('p_date')
    adj = adj.drop('temp_cum_split_factor', 'cum_split_factor', 'cum_split_factor_no_lag', 'row_num', 'p_split_date', 'p_split_factor')
    
    adj_pd = adj.toPandas()
    adj_pd['p_date'] = pd.to_datetime(adj_pd['p_date'])
    list_to_plot = sorted(adj_pd['ticker_region'].unique().tolist())
    
    columns = 3
    num_rows = math.ceil(len(list_to_plot) / columns)
    fig, axs = plt.subplots(nrows=num_rows, ncols=columns, figsize=(15, 5*num_rows))
    axs = axs.flatten()
    
    i = 0
    for t in list_to_plot:
        temp_df = adj_pd[adj_pd['ticker_region']==t]
        axs[i].plot(temp_df['p_date'], temp_df['split_adj_price'], label=t)
        
        imp_start_date = imp_df.loc[imp_df['ticker_region']==t, 'Implosion_Start_Date']
        imp_end_date = imp_df.loc[imp_df['ticker_region']==t, 'Implosion_End_Date']
        if imp_start_date is not None:
            # filtered_temp_df = temp_df[(temp_df['p_date'] >= imp_start_date) & (temp_df['p_date'] <= imp_end_date)]
            # vol = filtered_temp_df['split_adj_price'].std()
            imp_start_date = pd.to_datetime(imp_start_date)
            imp_end_date = pd.to_datetime(imp_end_date)
            #filtered_temp_df = temp_df[(temp_df['p_date'] >= imp_start_date) & (temp_df['p_date'] <= imp_end_date)]
            #print(filtered_temp_df.head())
            axs[i].axvspan(imp_start_date, imp_end_date, alpha=0.5, color='blue')
        axs[i].legend()
        #axs[i].text(0.5, -0.1, f'Volatility: {vol:.2f}', ha='center', transform=axs[i].transAxes)
        i+=1
        
    for i in range(len(list_to_plot), num_rows * columns):
        fig.delaxes(axs.flatten()[i])
    
        
    plt.tight_layout()
    plt.savefig(pic_name)
    
    
def sample_plots():
    imp_df = pd.read_csv('imploded_stocks_mv.csv', index_col=False)
    imps_only = imp_df.loc[imp_df['Implosion_Start_Date'].notnull()]
    imps_only = spark.createDataFrame(imps_only.tail(20))
    plot_prices(imps_only, 'all_implosions_mv_subplots.png')
    
    

#sample_plots()
    

In [8]:
def compare_methods():
    imp_df_price = pd.read_csv('imploded_stocks3.csv', index_col=False)
    imp_df_mv = pd.read_csv('imploded_stocks_mv.csv', index_col=False)
    imp_df_price = imp_df_price.loc[imp_df_price['Implosion_Start_Date'].notnull()]
    imp_df_mv =  imp_df_mv.loc[imp_df_mv['Implosion_Start_Date'].notnull()]
    print("Number imploded via Market Value: ", len(imp_df_mv))
    print("Number imploded via Price: ", len(imp_df_price))
    not_in_price = imp_df_mv[~imp_df_mv['ticker_region'].isin(imp_df_price['ticker_region'])]
    not_in_mv = imp_df_price[~imp_df_price['ticker_region'].isin(imp_df_mv['ticker_region'])]
    print("Number of stocks in price not in mv: ", len(not_in_mv))
    print("Number of stocks in mv not in price: ", len(not_in_price))
    print("Number of stocks in both: ", len(imp_df_price) - len(not_in_mv))
    plot_prices(spark.createDataFrame(not_in_price.head(20)), 'imploded_mv_only.png')
    plot_prices(spark.createDataFrame(not_in_mv.head(20)), 'imploded_price_only.png')
    
def compare_lengths():
    imp_df_price = pd.read_csv('imploded_stocks3.csv', index_col=False)
    imp_df_mv = pd.read_csv('imploded_stocks_mv.csv', index_col=False)
    imp_df_price = imp_df_price.loc[imp_df_price['Implosion_Start_Date'].notnull()]
    imp_df_mv =  imp_df_mv.loc[imp_df_mv['Implosion_Start_Date'].notnull()]
    imp_df_price['Implosion_Start_Date'] = pd.to_datetime(imp_df_price['Implosion_Start_Date'])
    imp_df_mv['Implosion_Start_Date'] = pd.to_datetime(imp_df_mv['Implosion_Start_Date'])
    imp_df_price['Implosion_End_Date'] = pd.to_datetime(imp_df_price['Implosion_End_Date'])
    imp_df_mv['Implosion_End_Date'] = pd.to_datetime(imp_df_mv['Implosion_End_Date'])
    imp_df_price['imp_length'] = imp_df_price['Implosion_End_Date']-  imp_df_price['Implosion_Start_Date']
    imp_df_mv['imp_length'] = imp_df_mv['Implosion_End_Date']-  imp_df_mv['Implosion_Start_Date']
    print("Average implosion length (price): ", imp_df_price['imp_length'].mean())
    print("Average implosion length (market val): ", imp_df_mv['imp_length'].mean())

def create_implosion_price_plots():
    imp_df_price = pd.read_csv('imploded_stocks3.csv', index_col=False)
    test_stocks = ['CTRM-US', 'MNMD-US', 'CLDX-US', 'PHIO-US', 'EGOV-US', 'RRD-US', 'GFED-US', 'PTEK-US', 
                   'VHC-US', 'STRZB-US', 'NXPL-US', 'TANH-US', 'CTIC-US', 'DTSX-US', 'BLNK-US', 'XRTX-US', 
                   'ATOS-US', 'APAGF-US', 'VTNR-US', 'UONE-US', 'PIXY-US', 'SLRX-US', 'METG-US', 'CLZR-US', 'FET-US']
    test_df = imp_df_price[imp_df_price['ticker_region'].isin(test_stocks)]
    plot_prices(spark.createDataFrame(test_df), 'imploded_test_104p_52lb.png')
    

    
# compare_methods()
# compare_lengths()
#create_implosion_price_plots()

In [9]:
#spark.stop()